In [36]:
import os
import cv2
import numpy as np
!pip install numpy opencv-python scikit-learn tensorflow

from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report


In [37]:
from google.colab import drive

# Google Drive'a bağlanma
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
# Veri setinin bulunduğu klasörün yolu
data_folder = '/content/drive/My Drive/lung_image_sets'


In [39]:
DATADIR = '/content/drive/My Drive/lung_image_sets'

CATEGORIES = ["lung_adenocarcinoma","lung_benign tissue","lung_squamous cell carcinoma"]
classes = os.listdir(DATADIR)

In [40]:
IMG_SIZE = 100


In [41]:
all_data =[]

def create_all_data():
    for category in CATEGORIES:
        path=os.path.join(DATADIR, category)
        class_num=CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img_array=cv2.imread(os.path.join(path,img))
                new_array=cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))
                all_data.append([new_array,class_num])
            except Exception as e:
                pass
create_all_data()

In [42]:
print("Toplam Görüntü Sayısı: ", len(all_data))

Toplam Görüntü Sayısı:  15000


In [43]:
X = []
y = []

for categories, label in all_data:
  X.append(categories)
  y.append(label)

In [44]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state =42)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.10, random_state=42)


In [45]:
x_train = np.array(x_train)
x_test = np.array(x_test)

y_train = np.array(y_train)
y_test = np.array(y_test)

x_val = np.array(x_val)
y_val = np.array(y_val)

In [46]:
print(x_train.shape)
print(x_test.shape)
print(x_val.shape)

print(y_train.shape)
print(y_test.shape)
print(y_val.shape)

(9450, 100, 100, 3)
(4500, 100, 100, 3)
(1050, 100, 100, 3)
(9450,)
(4500,)
(1050,)


In [47]:
import tensorflow as tf

train_yCl = tf.keras.utils.to_categorical(y_train, num_classes=3)
test_yCl = tf.keras.utils.to_categorical(y_test, num_classes=3)
valid_yCl = tf.keras.utils.to_categorical(y_val, num_classes=3)

In [48]:
model = Sequential([
    Flatten(input_shape=x_train[0].shape),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_val, y_val))

Epoch 1/10
296/296 [==============================] - 23s 74ms/step - loss: 239.6087 - accuracy: 0.6624 - val_loss: 72.5257 - val_accuracy: 0.7638
Epoch 2/10
296/296 [==============================] - 16s 55ms/step - loss: 94.7808 - accuracy: 0.7255 - val_loss: 117.1056 - val_accuracy: 0.6467
Epoch 3/10
296/296 [==============================] - 16s 55ms/step - loss: 43.6617 - accuracy: 0.7231 - val_loss: 33.0558 - val_accuracy: 0.7105
Epoch 4/10
296/296 [==============================] - 19s 65ms/step - loss: 21.6717 - accuracy: 0.7332 - val_loss: 11.1277 - val_accuracy: 0.7600
Epoch 5/10
296/296 [==============================] - 16s 55ms/step - loss: 14.2367 - accuracy: 0.7279 - val_loss: 9.7476 - val_accuracy: 0.7029
Epoch 6/10
296/296 [==============================] - 16s 55ms/step - loss: 3.4673 - accuracy: 0.7476 - val_loss: 5.0800 - val_accuracy: 0.7143
Epoch 7/10
296/296 [==============================] - 16s 55ms/step - loss: 5.1314 - accuracy: 0.7211 - val_loss: 1.3538 - va

In [49]:
# Test verisi üzerinde tahmin yapma
y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Accuracy
accuracy = accuracy_score(y_test, y_pred_classes)
print(f"Accuracy: {accuracy}")

# Precision
precision = precision_score(y_test, y_pred_classes, average='macro')
print(f"Precision: {precision}")

# Recall
recall = recall_score(y_test, y_pred_classes, average='macro')
print(f"Recall: {recall}")

# F1 Score
f1 = f1_score(y_test, y_pred_classes, average='macro')
print(f"F1 Score: {f1}")

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred_classes)
print(f"Confusion Matrix:\n{conf_matrix}")

# Classification Report
class_report = classification_report(y_test, y_pred_classes)
print(f"Classification Report:\n{class_report}")

141/141 [==============================] - 2s 14ms/step
Accuracy: 0.6395555555555555
Precision: 0.5617637796145686
Recall: 0.6447447963991602
F1 Score: 0.5384737395957054
Confusion Matrix:
[[   3   12 1509]
 [   0 1402   90]
 [  11    0 1473]]
Classification Report:
              precision    recall  f1-score   support

           0       0.21      0.00      0.00      1524
           1       0.99      0.94      0.96      1492
           2       0.48      0.99      0.65      1484

    accuracy                           0.64      4500
   macro avg       0.56      0.64      0.54      4500
weighted avg       0.56      0.64      0.53      4500

